# Configuración Inicial

In [1]:
import sys
sys.path.append('../src/')
from utils import configurar_ruta_datos, cargar_datos

configurar_ruta_datos('../datos/')
datos_hogares = cargar_datos('hogar')

Se han cargado 35 archivos de tipo 'hogar' desde la ruta '../datos/'.


# HOGAR

Tipo de hogar

In [2]:
for hogar in datos_hogares:
    personas = int(hogar['IX_TOT'])
    if personas == 1:
        # Hogar unipersonal
        hogar['TIPO_HOGAR'] = 'Unipersonal'
    elif personas > 1 and personas <= 5:
        # Hogar pequeño
        hogar['TIPO_HOGAR'] = 'Nuclear'
    elif personas > 5:
        # Hogar grande
        hogar['TIPO_HOGAR'] = 'Extendido'
    else:
         # Hogar vacío / Sin información
        hogar['TIPO_HOGAR'] = 'Sin información'

Material

In [3]:
for hogar in datos_hogares:
    material = int(hogar['IV4'] if hogar['IV4'] != '' else '0')
    if material >= 1 and material <= 4:
        # Durable
        hogar['MATERIAL_TECHUMBRE'] = 'Material durable'
    elif material >= 5 and material <= 7:
        # Precario
        hogar['MATERIAL_TECHUMBRE'] = 'Material precario'
    else:
        # Sin información
        hogar['MATERIAL_TECHUMBRE'] = 'No aplica'

Densidad del hogar

In [4]:
for hogar in datos_hogares:
    if not( hogar['II1'].isdigit() and int(hogar['II1']) > 0):
        hogar['DENSIDAD_HOGAR'] = 'Sin información'
    else:
        promedio = int(hogar['IX_TOT']) / int(hogar['II1'])
        if promedio < 1:
            hogar['DENSIDAD_HOGAR'] = 'Bajo'
        elif promedio > 1 and promedio <= 2:
            hogar['DENSIDAD_HOGAR'] = 'Medio'
        else:
            hogar['DENSIDAD_HOGAR'] = 'Alto'

Condición de Habitabilidad

In [5]:
for hogar in datos_hogares:
    puntaje = 0
    
    # 1. Puntaje por disponibilidad de agua (IV6)
    if hogar['IV6'] == '1':
        puntaje += 3
    elif hogar['IV6'] == '2':
        puntaje += 2
    elif hogar['IV6'] == '3':
        puntaje += 1
    
    # 2. Puntaje por existencia de baño (IV8)
    if hogar['IV8'] == '1':
        puntaje += 3

    
    # 3. Puntaje por tipo de desagüe (IV11)
    if hogar['IV8'] == '1':  # Solo si tiene baño
        if hogar['IV11'] == '1':
            puntaje += 3
        elif hogar['IV11'] == '2':
            puntaje += 2
        elif hogar['IV11'] == '3':
            puntaje += 1
    
    # 4. Puntaje por material del techo
    if hogar['MATERIAL_TECHUMBRE'] == 'Material durable':
        puntaje += 3
    elif hogar['MATERIAL_TECHUMBRE'] == 'Material precario':
        puntaje += 1
    
    # 5. Puntaje por material de pisos (IV3)
    if hogar['IV3'] == '1':
        puntaje += 3
    elif hogar['IV3'] == '2':
        puntaje += 2
    elif hogar['IV3'] == '3':
        puntaje += 1
    
    # Asignar clasificación según puntaje total
    if puntaje >= 12:
        hogar['CONDICION_DE_HABITABILIDAD'] = 'Buena'
    elif puntaje > 8:
        hogar['CONDICION_DE_HABITABILIDAD'] = 'Saludable'
    elif puntaje > 6:
        hogar['CONDICION_DE_HABITABILIDAD'] = 'Regular'
    else:
        hogar['CONDICION_DE_HABITABILIDAD'] = 'Insuficiente'
        

# Almacenamiento en nuevos archivos

In [6]:
# Define las columnas que deseas guardar para individuos y hogares
from collections import defaultdict
import csv

columnas_hogares = {'CODUSU', 'NRO_HOGAR', 'ANO4', 'TRIMESTRE', 'REGION', 'AGLOMERADO', 'PONDERA', 'TIPO_HOGAR', 'DENSIDAD_HOGAR', 'MATERIAL_TECHUMBRE', 'CONDICION_DE_HABITABILIDAD', 'II1', 'II7', 'IV1', 'IV3', 'IV4', 'IV6', 'IV8', 'IV9', 'IV11', 'IV12_3', 'ITF', 'IX_TOT'}

# HOGARES
# Agrupar datos por año y trimestre
hogares_por_trimestre = defaultdict(list)
for hogar in datos_hogares:
    anio = hogar.get('ANO4', 'NA')
    trimestre = hogar.get('TRIMESTRE', 'NA')
    if anio != 'NA' and trimestre != 'NA':
        key = f"A{anio}_T{trimestre}"
        hogares_por_trimestre[key].append(hogar)

# Crear archivos por cada grupo
for key, hogares in hogares_por_trimestre.items():
    with open(f'../datos/procesado/hog_{key}.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=list(columnas_hogares))
        writer.writeheader()
        for hogar in hogares:
            limpio = {k: v for k, v in hogar.items() if k in columnas_hogares}
            writer.writerow(limpio)